In [46]:
import tensorflow as tf

In [50]:
q = tf.FIFOQueue(3, "float")
init = q.enqueue_many(([0.,0.,0.],))

In [51]:
x = q.dequeue()
y = x+1
q_inc = q.enqueue([y])

In [ ]:
sess = tf.InteractiveSession()

init.run()
q_inc.run() # 0 0 1
print(sess.run(x)) # 0 1
q_inc.run() # 1 1
print(sess.run(x)) # 1
q_inc.run() # 2
print(sess.run(x))  
q_inc.run()
print(sess.run(x))

sess.close()

0.0
1.0
2.0


In [1]:
""" Example to demonstrate how to use queues
Author: Chip Huyen
Prepared for the class CS 20SI: "TensorFlow for Deep Learning Research"
cs20si.stanford.edu
"""
from __future__ import print_function
import tensorflow as tf
import numpy as np

N_SAMPLES = 1000
NUM_THREADS = 4


In [2]:
# Generating some simple data
# create 1000 random samples, each is a 1D array from the normal distribution (10, 1)
data = 10 * np.random.randn(N_SAMPLES, 4) + 1 
# create 1000 random labels of 0 and 1
target = np.random.randint(0, 2, size=N_SAMPLES) 

In [9]:
print(data[:10])

[[  5.76572093   2.28122173 -10.50475477   2.1237904 ]
 [  0.85616398   3.54233855 -15.21659621  23.54000259]
 [ -5.24711362  -2.15725609  19.13498679  -6.95176375]
 [  1.06855395 -13.16095694  -7.28249913  10.06801922]
 [-17.41371853 -22.06165674  -1.95095146  11.67654068]
 [-13.75901661 -16.44855327   7.02633504  -7.79550923]
 [  2.23275611  -4.92286737 -15.14727709   1.94594249]
 [ 14.38030647   9.05246385  -6.20700886   3.40025909]
 [-11.98857506   7.34758491  22.5011337    7.67725153]
 [-22.57474723  -7.58833143  20.62143722   7.78752915]]


In [8]:
queue = tf.FIFOQueue(capacity=50, dtypes=[tf.float32, tf.int32], shapes=[[4], []])

enqueue_op = queue.enqueue_many([data, target])
data_sample, label_sample = queue.dequeue()

# create ops that do something with data_sample and label_sample

# create NUM_THREADS to do enqueue
qr = tf.train.QueueRunner(queue, [enqueue_op] * NUM_THREADS)
with tf.Session() as sess:
    # create a coordinator, launch the queue runner threads.
    coord = tf.train.Coordinator()
    enqueue_threads = qr.create_threads(sess, coord=coord, start=True)
    for step in xrange(100): # do to 100 iterations
        if coord.should_stop():
            break
        data_batch, label_batch = sess.run([data_sample, label_sample])
        print(data_batch)
    coord.request_stop()
    coord.join(enqueue_threads)

[  5.76572084   2.28122163 -10.50475502   2.1237905 ]
[  0.85616398   3.54233861 -15.2165966   23.54000282]
[ -5.2471137   -2.15725613  19.13498688  -6.95176363]
[  1.06855392 -13.16095734  -7.28249931  10.06801891]
[-17.41371918 -22.06165695  -1.95095146  11.67654037]
[-13.75901699 -16.44855309   7.02633524  -7.79550934]
[  2.23275614  -4.9228673  -15.14727688   1.94594252]
[ 14.38030624   9.05246353  -6.20700884   3.40025902]
[-11.98857498   7.34758472  22.50113297   7.67725134]
[-22.57474709  -7.58833122  20.62143707   7.78752899]
[  5.6087203   -6.2940836    8.64904213  14.23479748]
[ 13.30307198  21.33951759  -4.77831793   3.08878899]
[ -2.79249334  16.43248558  -8.14454937  12.28921223]
[ 6.93180323  3.18304491  0.80269367 -1.32180691]
[-8.76774693 -2.42944455 -1.39939654  1.08156002]
[ 6.57062626 -9.16451359 -5.71520805 -3.74133849]
[  2.8332181    6.4105463   13.85227585   4.73533392]
[  4.69729376  13.29530239 -12.24832821 -16.95124626]
[  9.01655293  -5.60620928  -5.61538029 